# QuAM demonstration

In [1]:
import json
from quam.components import *
from quam.components.superconducting_qubits import *

from pathlib import Path
root_folder = Path("./output")
root_folder.mkdir(exist_ok=True)
for key in ["basic", "referenced", "referenced_multifile"]:
    (root_folder / key).mkdir(exist_ok=True)

2023-11-21 15:59:54,051 - qm - INFO     - Starting session: 3f9934f5-45b8-47fe-9d90-75ff6a44eba3


## Creating QuAM from scratch

The user defines his own QuAM by inheriting from `QuamRoot` (see `components/quam.py`).
Once defined, it can be instantiated as follows:

In [5]:
machine = QuAM()
machine

QuAM(mixers=[], qubits=[], resonators=[], local_oscillators=[], wiring={})

We see that five attributes are defined in quam, all empty so far. We can begin populating it with different `QuamComponent`s

In [6]:
num_qubits = 2
for idx in range(num_qubits):
    # Create qubit components
    transmon = Transmon(
        id=idx,
        xy=IQChannel(
            frequency_converter_up=FrequencyConverter(
                local_oscillator=LocalOscillator(power=10, frequency=6e9),
                mixer=Mixer(),
            ),
            opx_output_I=("con1", 3 * idx + 3),
            opx_output_Q=("con1", 3 * idx + 4),
        ),
        z=SingleChannel(opx_output=("con1", 3 * idx + 5)),
    )
    machine.qubits.append(transmon)

    # Create resonator components
    resonator = InOutIQChannel(
        opx_input_I=("con1", 1),
        opx_input_Q=("con1", 2),
        opx_output_I=("con1", 1),
        opx_output_Q=("con1", 2),
        id=idx, 
        frequency_converter_up=FrequencyConverter(
            mixer=Mixer(),
        ),
        frequency_converter_down=FrequencyConverter(
        ),
        local_oscillator=LocalOscillator(power=10, frequency=6e9),
    )
    machine.resonators.append(resonator)

machine

QuAM(mixers=[], qubits=[Transmon(id=0, xy=IQChannel(operations={}, id=None, opx_output_I=('con1', 3), opx_output_Q=('con1', 4), frequency_converter_up=FrequencyConverter(local_oscillator=LocalOscillator(frequency=6000000000.0, power=10), mixer=Mixer(local_oscillator_frequency=6000000000.0, intermediate_frequency=0.0, offset_I=0, offset_Q=0, correction_gain=0, correction_phase=0), gain=None), mixer=Mixer(local_oscillator_frequency=6000000000.0, intermediate_frequency=0.0, offset_I=0, offset_Q=0, correction_gain=0, correction_phase=0), local_oscillator=LocalOscillator(frequency=6000000000.0, power=10), intermediate_frequency=0.0), z=SingleChannel(operations={}, id=None, opx_output=('con1', 5), filter_fir_taps=None, filter_iir_taps=None, offset=0), resonator=None), Transmon(id=1, xy=IQChannel(operations={}, id=None, opx_output_I=('con1', 6), opx_output_Q=('con1', 7), frequency_converter_up=FrequencyConverter(local_oscillator=LocalOscillator(frequency=6000000000.0, power=10), mixer=Mixer(l

## Saving and loading quam

We can then save quam to a json file:

In [7]:
machine.save(root_folder / "basic" / "state.json")

Finally, we can also load quam from the same json file

In [8]:
loaded_quam = QuAM.load(root_folder / "basic" / "state.json")

Importantly, the `state.json` file only describes how QuAM is structured, and how the different QuAM components should be initialized. The components themselves are defined in their respective classes.

## Generating QUA config

We can also generate the qua config from quam. This recursively calls `QuamComponent.apply_to_config()` on all quam components.

In [9]:
qua_config = machine.generate_config()
json.dump(qua_config, open(root_folder / "basic" / "qua_config.json", "w"), indent=2)

In [10]:
root_folder / "basic" / "qua_config.json"

PosixPath('output/basic/qua_config.json')

## Quam using references

QuAM allows values to reference another part of QuAM by setting its value to a string starting with a colon: `"#/path/to/referenced/value"`.

As an example why this is useful, we previously hardcoded all output ports. However, grouping everything at a top-level `"wiring"`` makes more sense, so we can have all output ports reference to it.

Below we also add several more references

In [12]:
num_qubits = 2
machine = QuAM()
machine.wiring = {
    "qubits": [
        {
            "opx_output_I": ("con1", 3 * k + 3), 
            "opx_output_Q": ("con1", 3 * k + 4), 
            "opx_output_Z": ("con1", 3 * k + 5)
        }
        for k in range(num_qubits)
    ],
    "resonator": {
        "opx_output_I": ("con1", 1), 
        "opx_output_Q": ("con1", 2), 
        "opx_input_I": ("con1", 1), 
        "opx_input_Q": ("con1", 2)
    }
}

for idx in range(num_qubits):
    # Create qubit components
    transmon = Transmon(
        id=idx,
        xy=IQChannel(
            frequency_converter_up=FrequencyConverter(
                local_oscillator=LocalOscillator(power=10, frequency=6e9),
                mixer=Mixer(),
            ),
            opx_output_I=f"#/wiring/qubits/{idx}/opx_output_I",
            opx_output_Q=f"#/wiring/qubits/{idx}/opx_output_Q",
        ),
        z=SingleChannel(opx_output=f"#/wiring/qubits/{idx}/opx_output_Z",),
    )
    machine.qubits.append(transmon)

    # Create resonator components
    resonator = InOutIQChannel(
        opx_input_I="#/wiring/resonator/opx_input_I",
        opx_input_Q="#/wiring/resonator/opx_input_Q",
        opx_output_I="#/wiring/resonator/opx_output_I",
        opx_output_Q="#/wiring/resonator/opx_output_Q",
        id=idx, 
        frequency_converter_up=FrequencyConverter(
            mixer=Mixer(),
        ),
        local_oscillator=LocalOscillator(power=10, frequency=6e9),
        frequency_converter_down=FrequencyConverter(),
    )
    machine.resonators.append(resonator)

machine

QuAM(mixers=[], qubits=[Transmon(id=0, xy=IQChannel(operations={}, id=None, opx_output_I=('con1', 3), opx_output_Q=('con1', 4), frequency_converter_up=FrequencyConverter(local_oscillator=LocalOscillator(frequency=6000000000.0, power=10), mixer=Mixer(local_oscillator_frequency=6000000000.0, intermediate_frequency=0.0, offset_I=0, offset_Q=0, correction_gain=0, correction_phase=0), gain=None), mixer=Mixer(local_oscillator_frequency=6000000000.0, intermediate_frequency=0.0, offset_I=0, offset_Q=0, correction_gain=0, correction_phase=0), local_oscillator=LocalOscillator(frequency=6000000000.0, power=10), intermediate_frequency=0.0), z=SingleChannel(operations={}, id=None, opx_output=('con1', 5), filter_fir_taps=None, filter_iir_taps=None, offset=0), resonator=None), Transmon(id=1, xy=IQChannel(operations={}, id=None, opx_output_I=('con1', 6), opx_output_Q=('con1', 7), frequency_converter_up=FrequencyConverter(local_oscillator=LocalOscillator(frequency=6000000000.0, power=10), mixer=Mixer(l

We again save quam and the QUA config. The QUA config is identical to previous, but QuAM has changed significantly

In [13]:
machine.save(root_folder / "referenced" / "state.json")

In [14]:
qua_config = machine.generate_config()
json.dump(qua_config, open(root_folder / "referenced" / "qua_config.json", "w"), indent=2)

## Separating QuAM into multiple files

Finally, we can also separate parts of QuAM to be placed into a separate file.
This can be especially useful when combined with referencing because we can now have a separate file dedicated to the wiring of the experiment.

Here we point quam to a folder instead of a json file, and we specify that `"wiring"` should go to a separate `wiring.json` file in that folder:

In [15]:
machine.save(root_folder / "referenced_multifile" / "quam", content_mapping={"wiring.json": ["wiring"]})

It can subseuently be loaded as usual

In [16]:
machine = QuAM.load(root_folder / "referenced_multifile" / "quam")

Separating QuAM into multiple files allows the user to revert back to a previous QuAM but keep some parts the same.
For example, the user may want to revert all experimental settings, but the wiring of the setup should of course not change.